| Column | Encoded Values | Meaning |
|:--------|:----------------|:---------|
| **gender** | 0 = Male<br>1 = Female<br>2 = Other | Gender of the patient |
| **ever_married** | 0 = No<br>1 = Yes | Whether the patient was ever married |
| **work_type** | 0 = Children<br>1 = Govt_job<br>2 = Never_worked<br>3 = Private<br>4 = Self-employed | Type of employment |
| **Residence_type** | 0 = Rural<br>1 = Urban | Type of residence |
| **smoking_status** | 0 = Unknown<br>1 = Formerly smoked<br>2 = Never smoked<br>3 = Smokes | Smoking behavior |
| **hypertension** | 0 = No<br>1 = Yes | Whether the patient has hypertension |
| **heart_disease** | 0 = No<br>1 = Yes | Whether the patient has heart disease |
| **stroke** | 0 = No Stroke<br>1 = Stroke | Indicates whether the patient experienced a stroke |
| **risk_score** | Continuous value | Calculated risk score = `(age/100)*0.4 + hypertension*0.2 + heart_disease*0.2 + (avg_glucose_level/200)*0.2` |

In [ ]:
from sqlalchemy import create_engine, text, inspect
import pandas as pd
from dotenv import load_dotenv
import os
import boto3

In [ ]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

In [ ]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [ ]:
conn = engine.connect()

In [ ]:
inspector = inspect(engine)
tables = inspector.get_table_names()
print(tables)

In [ ]:
query = "SELECT * FROM stroke_data_processed;"
df_query = pd.read_sql(query, engine)
df_query.head()

In [ ]:
query = """
SELECT gender, COUNT(*) AS total FROM stroke_data_processed GROUP BY gender;
"""
df_query = pd.read_sql(query, engine)
df_query

In [ ]:
#AGE GROUPING
query = """
SELECT 
    CASE 
        WHEN age < 20 THEN 'Below 20'
        WHEN age BETWEEN 20 AND 40 THEN '20–40'
        WHEN age BETWEEN 40 AND 60 THEN '40–60'
        ELSE '60+'
    END AS age_group,
    COUNT(*) AS stroke_cases
FROM stroke_data_processed
WHERE stroke = 1
GROUP BY age_group
ORDER BY age_group;
"""
df_query = pd.read_sql(query, engine)
df_query

In [ ]:
#AGE GROUP WITH RISK SCORE
query = """
SELECT 
    CASE 
        WHEN age < 30 THEN 'Under 30'
        WHEN age BETWEEN 30 AND 50 THEN '30-50'
        WHEN age BETWEEN 51 AND 70 THEN '51-70'
        ELSE '70+'
    END AS age_group,
    COUNT(*) AS total,
    ROUND(AVG(risk_score::numeric), 3) AS avg_risk_score,
    SUM(stroke) AS stroke_cases
FROM stroke_data_processed
GROUP BY age_group
ORDER BY age_group;
"""
df_query = pd.read_sql(query, engine)
df_query

In [ ]:
#BASED ON HEART DISEASE HISTORY
query = """
SELECT 
    heart_disease,
    COUNT(*) AS total,
    SUM(stroke) AS stroke_count,
    ROUND(SUM(stroke)*100.0 / COUNT(*), 2) AS stroke_rate_percent
FROM stroke_data_processed
GROUP BY heart_disease;
"""
df_query = pd.read_sql(query, engine)
df_query

In [ ]:
#BASED ON BMI
query = """
SELECT 
    CASE 
        WHEN bmi < 18.5 THEN 'Underweight'
        WHEN bmi BETWEEN 18.5 AND 24.9 THEN 'Normal'
        WHEN bmi BETWEEN 25 AND 29.9 THEN 'Overweight'
        WHEN bmi BETWEEN 30 AND 34.9 THEN 'Obese'
        ELSE 'Severely Obese'
    END AS bmi_category,
    COUNT(*) AS total_patients,
    SUM(stroke) AS stroke_cases,
    ROUND(SUM(stroke)::numeric / COUNT(*) * 100, 2) AS stroke_rate_percent
FROM stroke_data_processed
GROUP BY bmi_category
ORDER BY bmi_category;
"""
df_query = pd.read_sql(query, engine)
df_query

In [ ]:
#AVG GLUCOSE LEVEL FOR STROKE VS NON-STROKE
query = """
SELECT 
    stroke,
    ROUND(AVG(avg_glucose_level)::numeric, 2) AS avg_glucose_level
FROM stroke_data_processed
GROUP BY stroke
ORDER BY stroke;
"""
df_query = pd.read_sql(query, engine)
df_query


In [ ]:
#to disconnect
conn.close()
engine.dispose()